In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.cm as cm

Coaches = pd.read_json('coaches/coaches.json')
Players = pd.read_json('players/players.json')
Events = pd.read_json('events/events_England.json')
Teams = pd.read_json('teams/teams.json')
Matches = pd.read_json('matches/matches_England.json')

In [2]:
# Add column 'Nation' on Teams
Nation = pd.DataFrame([x['name'] for x in Teams['area']],columns=['Nation'])
Teams = pd.merge(Teams, Nation, left_index=True, right_index=True)
Teams_England = Teams[((Teams['Nation'] == 'England') | (Teams['Nation'] == 'Wales')) & (Teams['type'] == 'club')]

In [3]:
# Generate England_Players DataFrame
England_Teams_ID_list = list(Teams_England['wyId']) # list teams Id
player_list =[]
for x in England_Teams_ID_list:
    player_list.append(Players[Players['currentTeamId'] ==x ])

England_Players = pd.concat(player_list) #DF with England Players info

In [4]:
# Generate England_Events DataFrame
England_Players_ID_list = list(England_Players['wyId'])
event_list =[]
for x in England_Players_ID_list:
    event_list.append(Events[Events['playerId'] == x ])

England_Events = pd.concat(event_list) #DF with England Events info

In [5]:
England_Events_Pass = England_Events[England_Events['eventName']=='Pass'] # sub-DataFrame with player 'passes'

In [6]:
count_total_pass = {} # dict containing the sums of pass for each player initialized to 0
count_accurate_pass = {} # dict containing the sums of accurate pass for each player initialized to 0
count_ratio_pass = {} # dict containing the ratio = count_accurate_pass/count_total_pass

# initialize
for x in England_Players_ID_list:
    count_total_pass.update({x:0})
    count_accurate_pass.update({x:0})
    count_ratio_pass.update({x:0})

# count
i = 0
for x in England_Events_Pass['playerId']:
    count_total_pass[x] += 1
    if str(England_Events_Pass['tags'].iloc[i]).find('1801') > 0:
        count_accurate_pass[x] += 1
    i +=1    

In [7]:
n = 100 # minimum threshold

for x in England_Players_ID_list: 
    if count_total_pass[x] < n:
        del count_ratio_pass[x]
    else:
        count_ratio_pass[x] = count_accurate_pass[x] / count_total_pass[x]

In [8]:
# PRINT - problem!?!?
player_pass =[x[1] for x in sorted((value,key) for (key,value) in count_ratio_pass.items())[-10:]]
for i in player_pass:
    lst_player = []
    lst_player.append(list(Players[Players['wyId']==i]['firstName']))
    lst_player.append(list(Players[Players['wyId']==i]['middleName']))
    lst_player.append(list(Players[Players['wyId']==i]['lastName']))
    
    #print(dict(Players[Players['wyId']==i][['firstName', 'middleName', 'lastName']]))
    print(u'{} {} {}'.format(lst_player[0][0],lst_player[1][0], lst_player[2][0]))

Moussa Sidi Yaya  Demb\u00e9l\u00e9
Victor  Nilsson Lindel\u00f6f
\u0130lkay  G\u00fcndo\u011fan
Mohamed Naser  Elsayed Elneny
Nicol\u00e1s Hern\u00e1n  Otamendi
Phil  Jones
Andreas  Christensen
Vincent  Kompany
Fabian  Delph
John  Stones
